<a href="https://colab.research.google.com/github/GuilhermeMoraesLS/Projeto_SE_Maxmalize/blob/main/prototipo_autoguard2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 🧠 Instalações necessárias ---
!pip install cmake
!pip install dlib-bin
!pip install git+https://github.com/ageitgey/face_recognition_models
!pip install face-recognition --no-deps

# --- 📦 Importações ---
import os
import numpy as np
import cv2
import face_recognition
from google.colab import files
from google.colab.patches import cv2_imshow
from collections import defaultdict

# --- 📁 Criar pasta 'database' e fazer upload das imagens ---
os.makedirs("database", exist_ok=True)
print("📂 Crie sua base de dados de rostos (várias fotos por pessoa)")
print("💡 Dica: use nomes tipo joao_1.jpg, joao_2.jpg... com e sem barba, óculos, etc.")

uploaded = files.upload()  # escolha suas imagens de referência
for filename in uploaded.keys():
    os.rename(filename, f"database/{filename}")
print("✅ Imagens adicionadas à pasta 'database'")

# --- 🧠 Função de pré-processamento da imagem ---
def preprocess_image(img_path):
    """Carrega e melhora a imagem para detecção mais precisa."""
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Normaliza tamanho (se imagem for muito grande)
    if max(img.shape) > 1500:
        img = cv2.resize(img, (0, 0), fx=0.75, fy=0.75)

    # Equaliza iluminação (em canal YCrCb)
    ycrcb = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
    img = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2RGB)

    return img

# --- 🧠 Carrega e calcula vetor médio de cada pessoa ---
def load_face_database(path="database"):
    encodings_dict = defaultdict(list)
    print("📥 Processando imagens do banco de dados...")
    for filename in os.listdir(path):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(path, filename)
            img = preprocess_image(img_path)
            encodings = face_recognition.face_encodings(img)
            if encodings:
                name = os.path.splitext(filename)[0].split("_")[0]  # joao_1.jpg -> joao
                encodings_dict[name].append(encodings[0])
            else:
                print(f"⚠ Nenhum rosto detectado em: {filename}")

    known_names, known_encodings = [], []
    for name, vectors in encodings_dict.items():
        mean_encoding = np.mean(vectors, axis=0)
        known_names.append(name)
        known_encodings.append(mean_encoding)

    print(f"✅ Banco de dados carregado: {len(known_names)} pessoas conhecidas.")
    return known_encodings, known_names

# --- 🔍 Função de reconhecimento (sem confidence) ---
def recognize_faces_in_image(image_path, known_encodings, known_names, tolerance=0.65):
    """
    Reconhece rostos com pré-processamento, fallback de detecção e vetores médios.
    """
    img = preprocess_image(image_path)

    # 🔎 Detecta rostos - tenta 'hog', depois 'cnn' se falhar
    face_locations = face_recognition.face_locations(img, model="hog")
    if len(face_locations) == 0:
        face_locations = face_recognition.face_locations(img, model="cnn")

    face_encodings = face_recognition.face_encodings(img, face_locations)
    results = []

    for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        if len(face_distances) > 0:
            best_idx = np.argmin(face_distances)
            best_distance = face_distances[best_idx]

            if best_distance < tolerance:
                name = known_names[best_idx]
            elif best_distance < tolerance + 0.05:
                name = f"{known_names[best_idx]} (?)"
            else:
                name = "Desconhecido"
        else:
            name = "Desconhecido"

        results.append((name, (top, right, bottom, left)))

    # 🖼 Desenha resultados
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    for name, (top, right, bottom, left) in results:
        cv2.rectangle(img_bgr, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(img_bgr, name, (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    return img_bgr, results

# --- 🧠 Carrega banco de dados ---
known_encodings, known_names = load_face_database("database")

# --- 📸 Upload da imagem de teste ---
print("\n📤 Faça upload da imagem para testar:")
uploaded_test = files.upload()
image_to_test = list(uploaded_test.keys())[0]

# --- 🔎 Executa reconhecimento ---
print("\n🔎 Analisando imagem...")
image_with_boxes, results = recognize_faces_in_image(
    image_to_test,
    known_encodings,
    known_names,
    tolerance=0.65  # tolerância mais flexível para variações como barba
)

# --- 📊 Resultados ---
print("\n📊 Resultados do reconhecimento:")
if not results:
    print("❌ Nenhum rosto detectado.")
else:
    for name, _ in results:
        print(f"- {name}")

# --- 📷 Exibe imagem final ---
print("\n🖼 Imagem com resultados:")
cv2_imshow(image_with_boxes)

# --- 💾 Salva resultado ---
cv2.imwrite("resultado.jpg", image_with_boxes)
print("\n✅ Resultado salvo como 'resultado.jpg'")
